In [ ]:
import torch

label = torch.randn(8, 10)
img = torch.randn(8, 1, 28, 28)
label = label.unsqueeze(2).unsqueeze(3)
print(label.shape)
label = label.expand(-1, -1, 28, 28)
print(label.shape)
img = torch.cat([img, label], dim=1)
print(img.shape)